# Paralelismo a Nível de Modelo (Model Parallelism)

O Paralelismo a Nível de Modelo, ou "Model Parallelism", refere-se à técnica de dividir um modelo de aprendizado profundo em partes e executar cada parte em um dispositivo diferente, geralmente GPUs. Isso é particularmente útil quando o modelo é muito grande para caber em uma única GPU.

Vamos criar um exemplo simples usando o PyTorch para demonstrar o paralelismo a nível de modelo. Neste exemplo, vamos dividir um modelo de rede neural simples em duas partes e executar cada parte em uma GPU diferente.

### Exemplo: Model Parallelism com PyTorch

In [ ]:
import torch
import torch.nn as nn

# Verificar a disponibilidade de GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

# Modelo de rede neural dividido em duas partes
class SplitNN(nn.Module):
    def __init__(self):
        super(SplitNN, self).__init__()
        self.part1 = nn.Sequential(
            nn.Linear(10, 50),
            nn.ReLU()
        ).cuda(0)  # Primeira parte do modelo na GPU 0
        
        self.part2 = nn.Sequential(
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 1)
        ).cuda(1)  # Segunda parte do modelo na GPU 1

    def forward(self, x):
        x = self.part1(x.cuda(0))
        return self.part2(x.cuda(1))

# Criando dados sintéticos
X = torch.rand(100, 10)

# Treinamento
model = SplitNN()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):
    outputs = model(X)
    loss = criterion(outputs, torch.ones(100, 1).cuda(1)) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}")

print("Training complete!")


Neste exemplo:

1. Criamos um modelo `SplitNN` que é dividido em duas partes. A primeira parte é colocada na GPU 0 e a segunda parte na GPU 1.
2. Durante a passagem para a frente (`forward`), os dados são passados pela primeira parte do modelo na GPU 0, depois são transferidos para a GPU 1 e passados pela segunda parte do modelo.
3. O treinamento é realizado normalmente, mas os gradientes são calculados e propagados de volta através de ambas as partes do modelo em suas respectivas GPUs.

Este código assume que você tem pelo menos duas GPUs disponíveis. Se você tiver apenas uma GPU ou estiver executando em uma máquina sem GPUs, precisará ajustar o código de acordo.

O paralelismo a nível de modelo é uma técnica avançada e pode ser complicado de implementar em modelos mais complexos. No entanto, pode ser uma solução valiosa quando se trabalha com modelos muito grandes que não cabem em uma única GPU.